# Foundations of Computer Science Project

## Import packages

Tutti i pacchetti utilizzati per il progetto.

In [1]:
import pandas as pd
import numpy as np
import re
import time

Con time decido di monitorare il tempo di esecuzione dell'intero codice.

In [2]:
start = time.time()

## Import datasets

Import dei dataset ed ispezione.

In [3]:
data_ps = pd.read_csv('googleplaystore.csv')
data_ur = pd.read_csv('googleplaystore_user_reviews.csv')

In [4]:
data_ps.head() #Check the googleplaystore dataset

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [5]:
data_ur.head() #Check the googleplaystore_user_reviews dataset

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


Durante l'esercizio 2 mi sono reso conto che il record 10472 possiede una codifica errata di tutti gli attributi dovuta ad un problema verificatisi durante la raccolta dei dati. Ho proceduto quindi all'**eliminazione** di tale record.

In [6]:
data_ps.loc[[10472]]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
10472,Life Made WI-Fi Touchscreen Photo Frame,1.9,19.0,3.0M,"1,000+",Free,0,Everyone,NaN,"February 11, 2018",1.0.19,4.0 and up,NaN


In [7]:
data_ps = data_ps.drop(10472)

## 1. Convert the app sizes to a number

Per evitare che 'Varies with device' crei problemi lo converto ad un forma da cui posso tornare indietro.

In [8]:
data_ps['Size'] = [re.sub('Varies with device', '99999999', size) for size in data_ps['Size']] 
data_ps['Size'] = [re.sub(',', '.', size) for size in data_ps['Size']] #Format 

In [9]:
sizes_to_num_re = re.compile('(?P<amount>\d*\.*\d*)(?P<unit>\w*\+*)') #

def unit_to_mult(unit):
    if unit == 'G':
        return 1000000000
    if unit == 'M':
        return 1000000 #Identifying the number of bytes into 1 Gb, Mb and Kb
    if unit == 'k':
        return 1000
    return 1

def to_numeric(elem): #Converting the strings into numbers
    m = sizes_to_num_re.search(elem)
    unit = m.group('unit')
    mult = unit_to_mult(unit)
    amount = float(m.group('amount'))
    return int(amount * mult)

In [10]:
data_ps['ExtSize'] = data_ps['Size'].apply(to_numeric) #Applying the function

Dopo aver convertito Size riporto 'Varies with device' alla sua forma originale per preservarne il significato.

In [11]:
data_ps['Size'] = [re.sub('99999999', 'Varies with device', size) for size in data_ps['Size']] #Reconverting the value '99999999' into 'Varies with device'
data_ps['ExtSize'] = data_ps['ExtSize'].apply(str)
data_ps['ExtSize'] = [re.sub('99999999', 'Varies with device', size) for size in data_ps['ExtSize']]
data_ps

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,ExtSize
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up,19000000
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,14000000
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up,8700000
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up,25000000
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up,2800000
5,Paper flowers instructions,ART_AND_DESIGN,4.4,167,5.6M,"50,000+",Free,0,Everyone,Art & Design,"March 26, 2017",1.0,2.3 and up,5600000
6,Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,3.8,178,19M,"50,000+",Free,0,Everyone,Art & Design,"April 26, 2018",1.1,4.0.3 and up,19000000
7,Infinite Painter,ART_AND_DESIGN,4.1,36815,29M,"1,000,000+",Free,0,Everyone,Art & Design,"June 14, 2018",6.1.61.1,4.2 and up,29000000
8,Garden Coloring Book,ART_AND_DESIGN,4.4,13791,33M,"1,000,000+",Free,0,Everyone,Art & Design,"September 20, 2017",2.9.2,3.0 and up,33000000
9,Kids Paint Free - Drawing Fun,ART_AND_DESIGN,4.7,121,3.1M,"10,000+",Free,0,Everyone,Art & Design;Creativity,"July 3, 2018",2.8,4.0.3 and up,3100000


## 2. Convert the number of installs to a number

In [12]:
data_ps.dtypes #Check the dataframe's columns dtypes

App                object
Category           object
Rating            float64
Reviews            object
Size               object
Installs           object
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
ExtSize            object
dtype: object

In [13]:
data_ps.groupby('Installs')['Installs'].size() #Check for some strange values

Installs
0                    1
0+                  14
1+                  67
1,000+             907
1,000,000+        1579
1,000,000,000+      58
10+                386
10,000+           1054
10,000,000+       1252
100+               719
100,000+          1169
100,000,000+       409
5+                  82
5,000+             477
5,000,000+         752
50+                205
50,000+            479
50,000,000+        289
500+               330
500,000+           539
500,000,000+        72
Name: Installs, dtype: int64

In [14]:
data_ps['NumInstalls'] = [re.sub(',', '', number) for number in data_ps['Installs']] #Remove the comma

In [15]:
data_ps['NumInstalls'] = [re.sub('\+*', '', number) for number in data_ps['NumInstalls']] #Remove the plus
data_ps['NumInstalls'] = data_ps['NumInstalls'].apply(float) #Converting to float

In [16]:
data_ps.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,ExtSize,NumInstalls
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up,19000000,10000.0
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,14000000,500000.0
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up,8700000,5000000.0
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up,25000000,50000000.0
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up,2800000,100000.0


In [17]:
data_ps.dtypes

App                object
Category           object
Rating            float64
Reviews            object
Size               object
Installs           object
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
ExtSize            object
NumInstalls       float64
dtype: object

## 3. Transform “Varies with device” into a missing value

In [18]:
data_psnan = data_ps.replace('Varies with device', np.nan)

Siccome a questo punto mi sono liberato definitivamente di 'Varies with device' posso convertire 'ExtSize' in float.

In [19]:
data_psnan['ExtSize'] = data_psnan['ExtSize'].apply(float) #Converting into float

In [20]:
data_psnan.dtypes #Checking

App                object
Category           object
Rating            float64
Reviews            object
Size               object
Installs           object
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
ExtSize           float64
NumInstalls       float64
dtype: object

## 4. Convert Current Ver and Android Ver into a dotted number (e.g. 4.0.3 or 4.2)

In [21]:
data_psnan.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,ExtSize,NumInstalls
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up,19000000.0,10000.0
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,14000000.0,500000.0
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up,8700000.0,5000000.0
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",NaN,4.2 and up,25000000.0,50000000.0
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up,2800000.0,100000.0


Decido di controllare la presenza di missing value ed eventualmente di eliminare le righe che li presentano.

In [22]:
data_psnan.isnull().any() #Checking for missing values

App               False
Category          False
Rating             True
Reviews           False
Size               True
Installs          False
Type               True
Price             False
Content Rating    False
Genres            False
Last Updated      False
Current Ver        True
Android Ver        True
ExtSize            True
NumInstalls       False
dtype: bool

In [23]:
data_nomis = data_psnan.dropna()
data_nomis.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,ExtSize,NumInstalls
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up,19000000.0,10000.0
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,14000000.0,500000.0
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up,8700000.0,5000000.0
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up,2800000.0,100000.0
5,Paper flowers instructions,ART_AND_DESIGN,4.4,167,5.6M,"50,000+",Free,0,Everyone,Art & Design,"March 26, 2017",1.0,2.3 and up,5600000.0,50000.0


In [24]:
data_nomis.groupby('Current Ver').size().head()

Current Ver
0.0.0.2    1
0.0.1      6
0.0.2      3
0.0.3      1
0.0.4      2
dtype: int64

In [25]:
pd.options.mode.chained_assignment = None

Sia per Current Version che per Android Version rimuovo tutti i caratteri che non sono numeri e poi procedo ad inserire un punto
come separatore tra ogni punto.

In [26]:
data_nomis['CurrVer'] = [re.sub('\\D', '', row) for row in data_nomis['Current Ver']] #Remove everything that is not a number from Current Ver

In [27]:
data_nomis['CurrVer'] = [re.sub(r'(?<!^)(?=(\d{1})+$)', r'.', numb) for numb in data_nomis['CurrVer']]

In [28]:
data_nomis.head() #Did it

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,ExtSize,NumInstalls,CurrVer
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up,19000000.0,10000.0,1.0.0
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,14000000.0,500000.0,2.0.0
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up,8700000.0,5000000.0,1.2.4
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up,2800000.0,100000.0,1.1
5,Paper flowers instructions,ART_AND_DESIGN,4.4,167,5.6M,"50,000+",Free,0,Everyone,Art & Design,"March 26, 2017",1.0,2.3 and up,5600000.0,50000.0,1.0


In [29]:
#Now on Android Ver
data_nomis.groupby('Android Ver')['Android Ver'].size()

Android Ver
1.0 and up          2
1.5 and up         14
1.6 and up         87
2.0 and up         27
2.0.1 and up        7
2.1 and up        113
2.2 and up        206
2.3 and up        564
2.3.3 and up      235
3.0 and up        211
3.1 and up          8
3.2 and up         31
4.0 and up       1106
4.0.3 - 7.1.1       2
4.0.3 and up     1184
4.1 - 7.1.1         1
4.1 and up       1921
4.2 and up        316
4.3 and up        193
4.4 and up        805
4.4W and up         6
5.0 - 6.0           1
5.0 - 8.0           2
5.0 and up        487
5.1 and up         17
6.0 and up         44
7.0 - 7.1.1         1
7.0 and up         39
7.1 and up          2
8.0 and up          5
Name: Android Ver, dtype: int64

In [30]:
data_nomis['AndroVer'] = [re.sub('\\D', '', row) for row in data_nomis['Android Ver']]
data_nomis['AndroVer'] = [re.sub(r'(?<!^)(?=(\d{1})+$)', r'.', numb) for numb in data_nomis['AndroVer']]

In [31]:
data_nomis.head() #Did it

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,ExtSize,NumInstalls,CurrVer,AndroVer
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up,19000000.0,10000.0,1.0.0,4.0.3
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,14000000.0,500000.0,2.0.0,4.0.3
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up,8700000.0,5000000.0,1.2.4,4.0.3
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up,2800000.0,100000.0,1.1,4.4
5,Paper flowers instructions,ART_AND_DESIGN,4.4,167,5.6M,"50,000+",Free,0,Everyone,Art & Design,"March 26, 2017",1.0,2.3 and up,5600000.0,50000.0,1.0,2.3


## 5. Remove the duplicates

In [32]:
data_nomis.groupby('App')['App'].size()

App
+Download 4 Instagram Twitter                                                                                                        1
- Free Comics - Comic Apps                                                                                                           1
.R                                                                                                                                   1
/u/app                                                                                                                               1
058.ba                                                                                                                               1
1. FC Köln App                                                                                                                       1
10 Best Foods for You                                                                                                                2
10 Minutes a Day Times Tables                      

Per affrontare il problema dei duplicati alla luce del fatto che si tratta di duplicati dovuti ad una raccolta di dati in periodi temporalmente diversi ho deciso di utilizzare le Reviews come discriminante per decidere quale sia il duplicato più recente e, quindi, quello contenente le informazioni di maggior interesse.

In [33]:
len(data_nomis)

7637

In [34]:
data_nomis["Reviews"] = pd.to_numeric(data_nomis["Reviews"]) #Utilizziamo il numero di reviews come discriminante per capire quale sia il duplicato più recente

In [35]:
data_nomis[data_nomis["App"] == "Coloring book moana"] #Prediamo questa applicazione come riferimento

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,ExtSize,NumInstalls,CurrVer,AndroVer
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,14000000.0,500000.0,2.0.0,4.0.3
2033,Coloring book moana,FAMILY,3.9,974,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,14000000.0,500000.0,2.0.0,4.0.3


Una volta deciso di affrontare il problema dei duplicati attraverso la discriminante delle Reviews ho ritenuto sufficiente ordinare il dataset in modo descrescente proprio in base a questa variabile imponendo poi l'eliminazione dei duplicati preservando i primi e, quindi, quelli con il numero di Reviews maggiore.

In [36]:
data_nomis = data_nomis.sort_values('Reviews', ascending = False) 
#Ordiniamo per il numero di reviews

In [37]:
data_nomis[data_nomis["App"] == "Coloring book moana"] #Andato a buon fine

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,ExtSize,NumInstalls,CurrVer,AndroVer
2033,Coloring book moana,FAMILY,3.9,974,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,14000000.0,500000.0,2.0.0,4.0.3
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,14000000.0,500000.0,2.0.0,4.0.3


In [38]:
data_nomis.drop_duplicates(inplace = True) #Eliminiamo i duplicati identici tra di loro

In [39]:
data_nomis.drop_duplicates(subset = 'App', keep = 'first', inplace = True) #Eliminiamo i duplicati che differiscono per il numero di reviews mantenendo solo quelli com le reviews più alte

In [40]:
data_nomis[data_nomis["App"] == "Coloring book moana"] #E infatti è andato a buon fine

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,ExtSize,NumInstalls,CurrVer,AndroVer
2033,Coloring book moana,FAMILY,3.9,974,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,14000000.0,500000.0,2.0.0,4.0.3


In [41]:
len(data_nomis) #Did it

6953

## 6. For each category, compute the number of apps

In [42]:
data_nomis.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,ExtSize,NumInstalls,CurrVer,AndroVer
1879,Clash of Clans,GAME,4.6,44893888,98M,"100,000,000+",Free,0,Everyone 10+,Strategy,"July 15, 2018",10.322.16,4.1 and up,98000000.0,1.000000e+08,1.0.3.2.2.1.6,4.1
1917,Subway Surfers,GAME,4.5,27725352,76M,"1,000,000,000+",Free,0,Everyone 10+,Arcade,"July 12, 2018",1.90.0,4.1 and up,76000000.0,1.000000e+09,1.9.0.0,4.1
1878,Clash Royale,GAME,4.6,23136735,97M,"100,000,000+",Free,0,Everyone 10+,Strategy,"June 27, 2018",2.3.2,4.1 and up,97000000.0,1.000000e+08,2.3.2,4.1
1966,Candy Crush Saga,GAME,4.4,22430188,74M,"500,000,000+",Free,0,Everyone,Casual,"July 5, 2018",1.129.0.2,4.1 and up,74000000.0,5.000000e+08,1.1.2.9.0.2,4.1
420,UC Browser - Fast Download Private & Secure,COMMUNICATION,4.5,17714850,40M,"500,000,000+",Free,0,Teen,Communication,"August 2, 2018",12.8.5.1121,4.0 and up,40000000.0,5.000000e+08,1.2.8.5.1.1.2.1,4.0


In [43]:
data_nomis.groupby('Category').size()

Category
ART_AND_DESIGN           55
AUTO_AND_VEHICLES        62
BEAUTY                   37
BOOKS_AND_REFERENCE     141
BUSINESS                221
COMICS                   46
COMMUNICATION           186
DATING                  122
EDUCATION                77
ENTERTAINMENT            54
EVENTS                   35
FAMILY                 1531
FINANCE                 255
FOOD_AND_DRINK           72
GAME                    803
HEALTH_AND_FITNESS      189
HOUSE_AND_HOME           49
LIBRARIES_AND_DEMO       60
LIFESTYLE               267
MAPS_AND_NAVIGATION      93
MEDICAL                 265
NEWS_AND_MAGAZINES      153
PARENTING                44
PERSONALIZATION         271
PHOTOGRAPHY             203
PRODUCTIVITY            221
SHOPPING                145
SOCIAL                  153
SPORTS                  221
TOOLS                   624
TRAVEL_AND_LOCAL        139
VIDEO_PLAYERS           110
WEATHER                  49
dtype: int64

## 7. For each category, compute the average rating

In [44]:
data_nomis.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,ExtSize,NumInstalls,CurrVer,AndroVer
1879,Clash of Clans,GAME,4.6,44893888,98M,"100,000,000+",Free,0,Everyone 10+,Strategy,"July 15, 2018",10.322.16,4.1 and up,98000000.0,1.000000e+08,1.0.3.2.2.1.6,4.1
1917,Subway Surfers,GAME,4.5,27725352,76M,"1,000,000,000+",Free,0,Everyone 10+,Arcade,"July 12, 2018",1.90.0,4.1 and up,76000000.0,1.000000e+09,1.9.0.0,4.1
1878,Clash Royale,GAME,4.6,23136735,97M,"100,000,000+",Free,0,Everyone 10+,Strategy,"June 27, 2018",2.3.2,4.1 and up,97000000.0,1.000000e+08,2.3.2,4.1
1966,Candy Crush Saga,GAME,4.4,22430188,74M,"500,000,000+",Free,0,Everyone,Casual,"July 5, 2018",1.129.0.2,4.1 and up,74000000.0,5.000000e+08,1.1.2.9.0.2,4.1
420,UC Browser - Fast Download Private & Secure,COMMUNICATION,4.5,17714850,40M,"500,000,000+",Free,0,Teen,Communication,"August 2, 2018",12.8.5.1121,4.0 and up,40000000.0,5.000000e+08,1.2.8.5.1.1.2.1,4.0


In [45]:
data_nomis.groupby('Category')[['Rating']].mean()

,Rating
Category,
ART_AND_DESIGN,4.381818
AUTO_AND_VEHICLES,4.146774
BEAUTY,4.291892
BOOKS_AND_REFERENCE,4.322695
BUSINESS,4.095475
COMICS,4.193478
COMMUNICATION,4.074194
DATING,3.963934
EDUCATION,4.368831


## 8. Create two dataframes: one for the genres and one bridging apps and genders. So that, for instance, the app Pixel Draw - Number Art Coloring Book appears twice in the bridging table, once for Art & Design, once for Creativity

In [46]:
data_nomis.head()
#Check

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,ExtSize,NumInstalls,CurrVer,AndroVer
1879,Clash of Clans,GAME,4.6,44893888,98M,"100,000,000+",Free,0,Everyone 10+,Strategy,"July 15, 2018",10.322.16,4.1 and up,98000000.0,1.000000e+08,1.0.3.2.2.1.6,4.1
1917,Subway Surfers,GAME,4.5,27725352,76M,"1,000,000,000+",Free,0,Everyone 10+,Arcade,"July 12, 2018",1.90.0,4.1 and up,76000000.0,1.000000e+09,1.9.0.0,4.1
1878,Clash Royale,GAME,4.6,23136735,97M,"100,000,000+",Free,0,Everyone 10+,Strategy,"June 27, 2018",2.3.2,4.1 and up,97000000.0,1.000000e+08,2.3.2,4.1
1966,Candy Crush Saga,GAME,4.4,22430188,74M,"500,000,000+",Free,0,Everyone,Casual,"July 5, 2018",1.129.0.2,4.1 and up,74000000.0,5.000000e+08,1.1.2.9.0.2,4.1
420,UC Browser - Fast Download Private & Secure,COMMUNICATION,4.5,17714850,40M,"500,000,000+",Free,0,Teen,Communication,"August 2, 2018",12.8.5.1121,4.0 and up,40000000.0,5.000000e+08,1.2.8.5.1.1.2.1,4.0


Per affrontare questo problema ho deciso, per quanto riguarda la creazione del dataframe contenente i generi, di splittare banalmente laddove le tuple contengano array di generi utilizzando come discriminante il simbolo ";".

In [47]:
data_nomis['GenresList'] = [genre.split(';') for genre in data_nomis['Genres']]
#Splitto gli array di generi

In [48]:
genres = data_nomis.GenresList.apply(pd.Series) #genres dataframe
genres.head()
#Così ottengo il dataframe dei generi

,0,1
1879,Strategy,NaN
1917,Arcade,NaN
1878,Strategy,NaN
1966,Casual,NaN
420,Communication,NaN


In [49]:
app = data_nomis['App']

Una volta creato il dataframe dei generi è stato sufficiente, una volta creato una dataframe contenente i nomi delle app, unirli facendo riferimento ai rispetti indici così da ottenere per ogni app tante ripetizioni quanti fossero i generi a loro assegnati, realizzando quindi a tutti gli effetti una tabella ponte.

In [50]:
#Bridgind table
genres_1 = pd.merge(genres, app.to_frame(), left_index = True, right_index = True)\
           .melt(id_vars = ['App'], value_name = "Genres_1")\
           .drop("variable", axis = 1)\
           .dropna() #per evitare che le app non aventi un secondo genere si ripresentino.
genres_1
#Così ho realizzato la tabella ponte

,App,Genres_1
0,Clash of Clans,Strategy
1,Subway Surfers,Arcade
2,Clash Royale,Strategy
3,Candy Crush Saga,Casual
4,UC Browser - Fast Download Private & Secure,Communication
5,8 Ball Pool,Sports
6,DU Battery Saver - Battery Charger & Battery Life,Tools
7,Cache Cleaner-DU Speed Booster (booster & clea...,Tools
8,Shadow Fight 2,Action
9,Pou,Casual


In [51]:
print(genres_1[genres_1['App'] == 'Pixel Draw - Number Art Coloring Book']) #Did it

                                         App      Genres_1
3707   Pixel Draw - Number Art Coloring Book  Art & Design
10660  Pixel Draw - Number Art Coloring Book    Creativity


## 9. For each genre, create a new column of the original dataframe. The new columns must have boolean values (True if the app has a given genre)

Per effettuare questo compito è stato sufficiente esplicitare una funzione ad hoc per assolvere al task. Come riferimento per i generi è stata utilizzata la colonna GenresList, creata nel punto precedente, con i generi presi singolarmente, così da preservare la distinzione tra i generi esplcitata nel punto precedente.

In [52]:
def create_dummies( df, colname ):
    col_dummies = pd.get_dummies(df[colname].apply(pd.Series).stack()).sum(level=0)
    col_dummies.drop(col_dummies.columns[0], axis=1, inplace=True)
    df = pd.concat([df, col_dummies.astype(bool)], axis=1)
    df.drop(colname, axis = 1, inplace = True )
    return df

In [53]:
genres_2 = create_dummies(data_nomis, 'GenresList')
genres_2.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,...,Simulation,Social,Sports,Strategy,Tools,Travel & Local,Trivia,Video Players & Editors,Weather,Word
1879,Clash of Clans,GAME,4.6,44893888,98M,"100,000,000+",Free,0,Everyone 10+,Strategy,...,False,False,False,True,False,False,False,False,False,False
1917,Subway Surfers,GAME,4.5,27725352,76M,"1,000,000,000+",Free,0,Everyone 10+,Arcade,...,False,False,False,False,False,False,False,False,False,False
1878,Clash Royale,GAME,4.6,23136735,97M,"100,000,000+",Free,0,Everyone 10+,Strategy,...,False,False,False,True,False,False,False,False,False,False
1966,Candy Crush Saga,GAME,4.4,22430188,74M,"500,000,000+",Free,0,Everyone,Casual,...,False,False,False,False,False,False,False,False,False,False
420,UC Browser - Fast Download Private & Secure,COMMUNICATION,4.5,17714850,40M,"500,000,000+",Free,0,Teen,Communication,...,False,False,False,False,False,False,False,False,False,False


## 10. For each genre, compute the average rating. What is the genre with highest average?

Anche per questo compito, al fine di preservare i generi distinti e di non utilizzare la codifica Genres originale, ho proceduto ad unire il dataframe originale senza missing value con il dataframe con i generi distinti così da poter calcolare il rating medio per ogni singolo genere.

In [54]:
genres_3 = genres_1.merge(data_nomis, on = 'App')
genres_3.head()

,App,Genres_1,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,ExtSize,NumInstalls,CurrVer,AndroVer,GenresList
0,Clash of Clans,Strategy,GAME,4.6,44893888,98M,"100,000,000+",Free,0,Everyone 10+,Strategy,"July 15, 2018",10.322.16,4.1 and up,98000000.0,1.000000e+08,1.0.3.2.2.1.6,4.1,[Strategy]
1,Subway Surfers,Arcade,GAME,4.5,27725352,76M,"1,000,000,000+",Free,0,Everyone 10+,Arcade,"July 12, 2018",1.90.0,4.1 and up,76000000.0,1.000000e+09,1.9.0.0,4.1,[Arcade]
2,Clash Royale,Strategy,GAME,4.6,23136735,97M,"100,000,000+",Free,0,Everyone 10+,Strategy,"June 27, 2018",2.3.2,4.1 and up,97000000.0,1.000000e+08,2.3.2,4.1,[Strategy]
3,Candy Crush Saga,Casual,GAME,4.4,22430188,74M,"500,000,000+",Free,0,Everyone,Casual,"July 5, 2018",1.129.0.2,4.1 and up,74000000.0,5.000000e+08,1.1.2.9.0.2,4.1,[Casual]
4,UC Browser - Fast Download Private & Secure,Communication,COMMUNICATION,4.5,17714850,40M,"500,000,000+",Free,0,Teen,Communication,"August 2, 2018",12.8.5.1121,4.0 and up,40000000.0,5.000000e+08,1.2.8.5.1.1.2.1,4.0,[Communication]


In [55]:
genres_3.groupby('Genres_1')[['Rating']].mean()

,Rating
Genres_1,
Action,4.236918
Action & Adventure,4.279268
Adventure,4.190278
Arcade,4.268263
Art & Design,4.378947
Auto & Vehicles,4.146774
Beauty,4.291892
Board,4.290385
Books & Reference,4.320979


In [56]:
genres_3.groupby('Genres_1')['Rating'].mean().idxmax()

'Events'

In [57]:
genres_3.groupby('Genres_1')['Rating'].mean().max()

4.474285714285714

## 11. For each app, compute the approximate income, obtain as a product of number of installs and price.

Banalmente per questo compito ho proceduto ad **eliminare** tutti i caratteri speciali da Price e a **convertirlo** in un attributo numerico per poi procedere al calcolo dei guadagni approssimativi.

In [58]:
data_nomis['NumPrice'] = [re.sub('\$','', price) for price in data_nomis['Price']]

In [59]:
data_nomis['NumPrice'] = data_nomis['NumPrice'].apply(float)

In [60]:
data_nomis['Income'] = data_nomis['NumPrice'] * data_nomis['NumInstalls']
data_nomis.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,ExtSize,NumInstalls,CurrVer,AndroVer,GenresList,NumPrice,Income
1879,Clash of Clans,GAME,4.6,44893888,98M,"100,000,000+",Free,0,Everyone 10+,Strategy,"July 15, 2018",10.322.16,4.1 and up,98000000.0,1.000000e+08,1.0.3.2.2.1.6,4.1,[Strategy],0.0,0.0
1917,Subway Surfers,GAME,4.5,27725352,76M,"1,000,000,000+",Free,0,Everyone 10+,Arcade,"July 12, 2018",1.90.0,4.1 and up,76000000.0,1.000000e+09,1.9.0.0,4.1,[Arcade],0.0,0.0
1878,Clash Royale,GAME,4.6,23136735,97M,"100,000,000+",Free,0,Everyone 10+,Strategy,"June 27, 2018",2.3.2,4.1 and up,97000000.0,1.000000e+08,2.3.2,4.1,[Strategy],0.0,0.0
1966,Candy Crush Saga,GAME,4.4,22430188,74M,"500,000,000+",Free,0,Everyone,Casual,"July 5, 2018",1.129.0.2,4.1 and up,74000000.0,5.000000e+08,1.1.2.9.0.2,4.1,[Casual],0.0,0.0
420,UC Browser - Fast Download Private & Secure,COMMUNICATION,4.5,17714850,40M,"500,000,000+",Free,0,Teen,Communication,"August 2, 2018",12.8.5.1121,4.0 and up,40000000.0,5.000000e+08,1.2.8.5.1.1.2.1,4.0,[Communication],0.0,0.0


## 12. For each app, compute its minimum and maximum Sentiment_polarity

Per questo compito ho proceduto ad individuare per ogni App la **massima** e la **minima** sentiment_polarity singolarmente per poi concludere creando un dataframe che sintetizzi entrambi gli aspetti.

In [61]:
data_ur.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


In [62]:
data_max = data_ur.groupby('App')['Sentiment_Polarity'].max()
data_max = pd.DataFrame(data_max)
data_max = data_max.rename(columns = {'Sentiment_Polarity':'Max_Sentiment_Polarity'})
data_max.head()

,Max_Sentiment_Polarity
App,
10 Best Foods for You,1.000000
104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,0.910000
11st,1.000000
1800 Contacts - Lens Store,0.838542
1LINE – One Line with One Touch,1.000000


In [63]:
data_min = data_ur.groupby('App')['Sentiment_Polarity'].min()
data_min = pd.DataFrame(data_min)
data_min = data_min.rename(columns = {'Sentiment_Polarity':'Min_Sentiment_Polarity'})
data_min.head()

,Min_Sentiment_Polarity
App,
10 Best Foods for You,-0.8000
104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,-0.1125
11st,-1.0000
1800 Contacts - Lens Store,-0.3000
1LINE – One Line with One Touch,-0.8250


In [64]:
data_sentiment_polarity = data_max.merge(data_min, left_index = True, right_index = True)
data_sentiment_polarity.head()

,Max_Sentiment_Polarity,Min_Sentiment_Polarity
App,,
10 Best Foods for You,1.000000,-0.8000
104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,0.910000,-0.1125
11st,1.000000,-1.0000
1800 Contacts - Lens Store,0.838542,-0.3000
1LINE – One Line with One Touch,1.000000,-0.8250


In [65]:
end = time.time()
print ("##Il tempo di esecuzione del codice è stato di "+str(end-start)+" secondi##")

##Il tempo di esecuzione del codice è stato di 7.591020107269287 secondi##
